# Loading import library

In [1]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
from rdkit.Chem.Scaffolds.MurckoScaffold import MakeScaffoldGeneric
from rdkit.Chem.Scaffolds.MurckoScaffold import MurckoScaffoldSmiles


from scripts import metrics
import importlib as imp
imp.reload(metrics)
import time

# Funktion for calculate metrics

In [2]:
def calculate_metrics(output_set, recall_set, number, scaffold_type, type_cluster, generator, receptor, save=True):

    mt = metrics.Metrics(type_cluster, scaffold_type, generator, number, receptor, save)     
    mt.load(output_set, recall_set)
    result = mt.calculate_metrics_with_return()
    display(result)
    return result

## Count metrics 
- Need the change parameters depend on which type of scaffold/cluster/generator/target you have

In [32]:
type_cluster = 'sim' #options: 'dis'|'sim' 
numbers = [0,1,2,3,4] #options: [0]|[0,1]|[0,1,2]|[0,1,2,3]|[0,1,2,3,4] -> all combinations 0-4, depends on which calculation you have
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generator = 'DrugEx_GT' #options: 'Molpher'|'DrugEx'|'REINVENT'|'addcarbon'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
save_options = True

In [33]:
for number in numbers:
    before = time.perf_counter()
    a0 = calculate_metrics(f"data/output_sets/{receptor}/cOS_{generator}_{type_cluster}_{number}_one_column.csv", \
                                                                f"data/input_recall_sets/{receptor}/cRS_{receptor}_{type_cluster}_{number}.csv",\
                                   number, scaffold_type, type_cluster,generator,receptor,save_options)
    a0
    after = time.perf_counter()
    print('Path found after', (after - before)/60, 'minutes.')

data/output_sets/Glucocorticoid_receptor/cOS_DrugEx_GT_sim_0_one_column.csv


[23:51:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:51:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:51:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:51:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:51:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:51:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:51:25] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:52:51] WARNING: could not find number of expected rings. Switching to an approximate ring finding algorithm.
[23:52:51] WARNING: could not find number of expected rings. Switching to an approximate ring fi

add_columns_same_like_input_function


/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

UASo:  81
SSr:  486
SSo:  893470
UASr:  115
TUPOR_1:  0.0003831276298904953
TUPOR_2:  2.0371029343061946
Save


...

add_columns_same_like_input_function


/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

UASo:  25
SSr:  42
SSo:  899315
UASr:  40
TUPOR_1:  2.9188882649572174e-05
TUPOR_2:  0.39799406353724776
Save


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_4,sim,murcko,545405,899315,25/40,0.625,0.011459,0.000029,0.397994,0.606467,0.000822,739


Path found after 4.956600099553665 minutes.


In [97]:
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
save_options = True
for generator in [  'DrugEx_GT_epsilon_0.1_desired']:  #DrugEx_GT_1, DrugEx_RNN, REINVENT, GB_GA  ,'DrugEx_RNN','DrugEx_GT','DrugEx_GT_1','REINVENT','GB_GA', 'addcarbon'
    for scaffold_type in ['csk']:
        for type_cluster in ['sim']:
            for number in [2,3,4]:
                before = time.perf_counter()
                a0 = calculate_metrics(f"data/output_sets/{receptor}/cOS_{generator}_{type_cluster}_{number}_one_column.csv", \
                                                                f"data/input_recall_sets/{receptor}/cRS_{receptor}_{type_cluster}_{number}.csv",\
                                   number, scaffold_type, type_cluster,generator,receptor,save_options)
                a0
                after = time.perf_counter()
                print('Path found after', (after - before)/60, 'minutes.')

data/output_sets/Glucocorticoid_receptor/cOS_DrugEx_GT_epsilon_0.1_desired_sim_2_one_column.csv


[09:50:08] Explicit valence for atom # 5 C, 5, is greater than permitted
[09:50:20] Explicit valence for atom # 3 C, 5, is greater than permitted
[09:50:33] Explicit valence for atom # 3 C, 5, is greater than permitted
[09:50:38] Explicit valence for atom # 11 C, 5, is greater than permitted
[09:51:14] Explicit valence for atom # 4 C, 5, is greater than permitted
[09:51:22] Explicit valence for atom # 20 C, 5, is greater than permitted
[09:51:25] Explicit valence for atom # 1 C, 5, is greater than permitted
[09:51:50] Explicit valence for atom # 31 C, 5, is greater than permitted
[09:52:19] Explicit valence for atom # 31 C, 5, is greater than permitted
[09:52:29] Explicit valence for atom # 29 C, 5, is greater than permitted


add_columns_same_like_input_function


/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[1][x] = [df_generated[0].value_counts()[y] if y in df_generated[0].unique() else 0][0]
/home/filv/phd_projects/iga_2023/git_reccal/new/recall_metrics/scripts/metrics.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See 

UASo:  16
SSr:  135
SSo:  774747
UASr:  37
TUPOR_1:  7.535153847433855e-05
TUPOR_2:  0.19662067527248556
Save


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_epsilon_0.1_desired_2,sim,csk,96547,774747,16/37,0.432432,0.04479,0.000075,0.196621,0.124617,0.001222,947


Path found after 4.438127164170146 minutes.
data/output_sets/Glucocorticoid_receptor/cOS_DrugEx_GT_epsilon_0.1_desired_sim_3_one_column.csv


Process ForkPoolWorker-2790:
Process ForkPoolWorker-2787:
Process ForkPoolWorker-2792:
Process ForkPoolWorker-2793:
Process ForkPoolWorker-2788:
Process ForkPoolWorker-2791:
Process ForkPoolWorker-2789:
Process ForkPoolWorker-2800:
Process ForkPoolWorker-2785:
Process ForkPoolWorker-2786:
Process ForkPoolWorker-2797:
Process ForkPoolWorker-2795:
Process ForkPoolWorker-2798:
Process ForkPoolWorker-2799:
Process ForkPoolWorker-2794:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/filv/.conda/envs/drugex/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/filv/.cond

KeyboardInterrupt: 

## Calculate acarage value and save csv
- Do this step after calculate all metrics for each cluster

In [86]:
def avarage_value(numbers, type_cluster, scaffold_type, generator, receptor):

    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/"
    link = {}
    for x in numbers:
        link[f'link_{x}'] = f'{path}metrics_cluster_{x}_{type_cluster}_{generator}.csv'
        
    print("generator: ", generator)

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)
        

    a = df.mean(numeric_only=True)
    df.loc[len(df)] = [f"{generator}_mean", type_cluster, scaffold_type, a['USo'], a['SSo'], '-',\
                       a['TUPOR'],a['TUPOR_unique'],a['TUPOR_1'],a['TUPOR_2'], a['SESY'], a['ASER'], a['CwASo']]
    df = df.round(7)
    dff = df.copy()
    dff['SSo'] = dff['SSo'].apply(lambda x : "{:,}".format(x))
    dff['USo'] = dff['USo'].apply(lambda x : "{:,}".format(x))
    dff['CwASo'] = dff['CwASo'].apply(lambda x : "{:,}".format(x))


    display(df)

    mean = pd.DataFrame()
    mean = df.loc[len(numbers)].to_frame().T
                                        
    display(mean)
    dff.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean_with_coma.csv", index = False)
    df.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.csv", index = False)
    dff.to_html(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}/df_all_clusters_with_mean.html", index = False)
    mean.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}_mean_{scaffold_type}_{type_cluster}.csv", index=False) #float_format='%.7f'

    return mean

In [90]:
type_cluster = 'sim' #options: 'dis'|'sim' 
number = [0,1,2,3,4] #options: 1-5
scaffold_type = 'csk' #options: 'csk'|'murcko'
generator = 'DrugEx_GT_epsilon_0.1_desired' #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'``

In [91]:
df = avarage_value(number, type_cluster, scaffold_type, generator, receptor)

generator:  DrugEx_GT_epsilon_0.1_desired


FileNotFoundError: [Errno 2] No such file or directory: 'data/results/Glucocorticoid_receptor/csk_scaffolds/sim/DrugEx_GT_epsilon_0.1_desired/metrics_cluster_0_sim_DrugEx_GT_epsilon_0.1_desired.csv'

In [ ]:
#'Molpher','REINVENT','GB_GA', 'addcarbon' ,'DrugEx_GT','DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6',

In [68]:
for generator in [ 'DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_RNN_epsilon_0.6',  'DrugEx_GT_epsilon_0.1_desired','DrugEx_GT_epsilon_0.6_desired']:
    for scaffold_type in ['csk', 'murcko']:
        for type_cluster in ['dis']:
            number = [0,1,2,3,4]
            receptor = 'Glucocorticoid_receptor'
            avarage_value(number, type_cluster, scaffold_type, generator, receptor)

generator:  DrugEx_GT


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_0,dis,csk,204256.0,887001.0,32/40,0.800000,0.039166,0.000162,0.828998,0.230277,0.009798,8691.0
1,DrugEx_GT_1,dis,csk,246456.0,903812.0,17/23,0.739130,0.029990,0.000051,0.552071,0.272685,0.003101,2803.0
2,DrugEx_GT_2,dis,csk,211132.0,889336.0,14/40,0.350000,0.016577,0.000073,0.386375,0.237404,0.001056,939.0
3,DrugEx_GT_3,dis,csk,265790.0,892985.0,27/37,0.729730,0.027455,0.000187,1.344282,0.297642,0.008449,7545.0
4,DrugEx_GT_4,dis,csk,282960.0,886398.0,37/43,0.860465,0.030409,0.000295,1.941935,0.319225,0.021750,19279.0
5,DrugEx_GT_mean,dis,csk,242118.8,891906.4,-,0.695865,0.028720,0.000154,1.010732,0.271447,0.008831,7851.4


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_GT_mean,dis,csk,242118.8,891906.4,-,0.695865,0.02872,0.000154,1.010732,0.271447,0.008831,7851.4


generator:  DrugEx_GT


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_0,dis,murcko,493954.0,887056.0,37/92,0.402174,0.008142,0.000082,0.438161,0.556847,0.002210,1960.0
1,DrugEx_GT_1,dis,murcko,544621.0,903889.0,15/34,0.441177,0.008101,0.000031,0.492553,0.602531,0.000427,386.0
2,DrugEx_GT_2,dis,murcko,513152.0,889392.0,9/64,0.140625,0.002740,0.000029,0.235802,0.576969,0.000429,382.0
3,DrugEx_GT_3,dis,murcko,543452.0,893054.0,39/76,0.513158,0.009443,0.000132,0.940928,0.608532,0.004031,3600.0
4,DrugEx_GT_4,dis,murcko,567567.0,886439.0,43/98,0.438775,0.007731,0.000150,0.871482,0.640278,0.003522,3122.0
5,DrugEx_GT_mean,dis,murcko,532549.2,891966.0,-,0.387182,0.007231,0.000085,0.595785,0.597031,0.002124,1890.0


...

,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_epsilon_0.6_desired_0,dis,csk,131092.0,531904.0,27/40,0.675000,0.051491,0.000228,0.748616,0.246458,0.007024,3736.0
1,DrugEx_GT_epsilon_0.6_desired_1,dis,csk,134840.0,535341.0,16/23,0.695652,0.051591,0.000082,0.479947,0.251877,0.006177,3307.0
2,DrugEx_GT_epsilon_0.6_desired_2,dis,csk,141333.0,559072.0,14/40,0.350000,0.024764,0.000116,0.411431,0.252799,0.000900,503.0
3,DrugEx_GT_epsilon_0.6_desired_3,dis,csk,127852.0,483275.0,23/37,0.621622,0.048620,0.000295,1.017825,0.264553,0.010764,5202.0
4,DrugEx_GT_epsilon_0.6_desired_4,dis,csk,129439.0,439692.0,36/43,0.837209,0.064680,0.000579,1.742432,0.294386,0.008033,3532.0
5,DrugEx_GT_epsilon_0.6_desired_mean,dis,csk,132911.2,509856.8,-,0.635897,0.048229,0.000260,0.880050,0.262015,0.006580,3256.0


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_GT_epsilon_0.6_desired_mean,dis,csk,132911.2,509856.8,-,0.635897,0.048229,0.00026,0.88005,0.262015,0.00658,3256.0


generator:  DrugEx_GT_epsilon_0.6_desired


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_epsilon_0.6_desired_0,dis,murcko,286483.0,531965.0,16/92,0.173913,0.006071,0.000059,0.183245,0.538537,0.001026,546.0
1,DrugEx_GT_epsilon_0.6_desired_1,dis,murcko,293935.0,535366.0,14/34,0.411765,0.014009,0.000048,0.418901,0.549036,0.000723,387.0
2,DrugEx_GT_epsilon_0.6_desired_2,dis,murcko,313410.0,559119.0,7/64,0.109375,0.003490,0.000036,0.178180,0.560543,0.000515,288.0
3,DrugEx_GT_epsilon_0.6_desired_3,dis,murcko,271328.0,483290.0,29/76,0.381579,0.014063,0.000181,0.645495,0.561419,0.007397,3575.0
4,DrugEx_GT_epsilon_0.6_desired_4,dis,murcko,247092.0,439698.0,19/98,0.193878,0.007846,0.000134,0.337971,0.561958,0.001171,515.0
5,DrugEx_GT_epsilon_0.6_desired_mean,dis,murcko,282449.6,509887.6,-,0.254102,0.009096,0.000092,0.352758,0.554299,0.002167,1062.2


,name,type_cluster,scaffold,USo,SSo,TUPOR_,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
5,DrugEx_GT_epsilon_0.6_desired_mean,dis,murcko,282449.6,509887.6,-,0.254102,0.009096,0.000092,0.352758,0.554299,0.002167,1062.2


In [37]:

df.to_csv(f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/{generator}_mean_{scaffold_type}_{type_cluster}.csv", index=False)

# Connect all mean value to one pandas DataFrame

In [69]:
def connect_mean_value(reeptor, scaffold_type, type_cluster, generators_name_list):
    path = f"data/results/{receptor}/{scaffold_type}_scaffolds/{type_cluster}/"
    link = {}
    for generator in generators_name_list:
        link[f'link_{generator}'] = f'{path}{generator}_mean_{scaffold_type}_{type_cluster}.csv'

    df = pd.DataFrame()
    for x,y in link.items():
        a = pd.read_csv(y)
        df = pd.concat([df, a], ignore_index=True)

    df = df.drop(columns = ['TUPOR_'])

    dff = df.copy()
    dff['SSo'] = dff['SSo'].apply(lambda x : "{:,}".format(x))
    dff['USo'] = dff['USo'].apply(lambda x : "{:,}".format(x))
    dff['CwASo'] = dff['CwASo'].apply(lambda x : "{:,}".format(x))

    
    dff.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}_with_coma.csv", index = False)
    df.to_csv(f"{path}/mean_{scaffold_type}_{type_cluster}.csv", index = False)
    dff.to_html(f"{path}/mean_{scaffold_type}_{type_cluster}.html", index = False)

    
    display(df)
    return df

In [71]:
type_cluster = 'sim' #options: 'dis'|'sim' 
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generators_name_list = [ 'DrugEx_GT_desired'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)


,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_GT_desired_mean,sim,murcko,254849.2,432504.4,0.599833,0.029288,0.000366,0.827606,0.592944,0.005301,2121.2


In [89]:
type_cluster = 'sim' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6', 'DrugEx_GT_desired', 'DrugEx_GT_epsilon_0.1_desired','DrugEx_GT_epsilon_0.6_desired'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'
df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)


FileNotFoundError: [Errno 2] No such file or directory: 'data/results/Glucocorticoid_receptor/csk_scaffolds/sim/DrugEx_GT_epsilon_0.1_desired_mean_csk_sim.csv'

In [37]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['Molpher', 'DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6','DrugEx_RNN','DrugEx_RNN_epsilon_0.1','DrugEx_RNN_epsilon_0.6','REINVENT', 'addcarbon','GB_GA'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

In [38]:
df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_2,SESY,ASER,CwASo
0,Molpher_mean,dis,csk,100068.0,823977.8,0.516682,0.360385,0.124815,0.004589,3041.2
1,DrugEx_GT_mean,dis,csk,242118.8,891906.4,0.695865,1.010732,0.271447,0.008831,7851.4
2,DrugEx_GT_epsilon_0.1_mean,dis,csk,140428.0,809233.2,0.545466,0.511284,0.172969,0.004915,4063.2
3,DrugEx_GT_epsilon_0.6_mean,dis,csk,267648.0,907474.4,0.679649,1.055034,0.294816,0.007886,7186.4
4,DrugEx_RNN_mean,dis,csk,110595.8,868587.2,0.665922,0.450804,0.127252,0.018142,16078.4
5,DrugEx_RNN_epsilon_0.1_mean,dis,csk,72000.6,797636.8,0.387368,0.220224,0.090468,0.005051,3941.0
6,DrugEx_RNN_epsilon_0.6_mean,dis,csk,87100.4,891065.4,0.620167,0.351509,0.097631,0.017469,15682.2
7,REINVENT_mean,dis,csk,169670.2,964580.2,0.559826,0.516835,0.175880,0.013492,13023.4
8,addcarbon_mean,dis,csk,856.8,1000000.0,0.033403,0.000148,0.000857,0.001827,1827.0
9,GB_GA_mean,dis,csk,2710.0,9998.0,0.197592,0.280268,0.271057,0.010821,108.2


In [25]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['DrugEx_epsilon_0.6', 'DrugEx_GT_1'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,DrugEx_epsilon_0.6_mean,dis,csk,4889.8,9993.8,0.354260,0.710789,0.007132,0.948241,0.489278,0.008363,83.6
1,DrugEx_GT_1_mean,dis,csk,254849.2,908272.6,0.700922,0.027669,0.000153,1.025833,0.280542,0.009991,9050.0


In [96]:
type_cluster = 'dis' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['Molpher','REINVENT','DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6','DrugEx_RNN','DrugEx_RNN_epsilon_0.1','DrugEx_RNN_epsilon_0.6', 'GB_GA', 'addcarbon'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,Molpher_mean,dis,csk,100068.0,823977.8,0.516682,0.053819,0.000158,0.360385,0.124815,0.004589,3041.2
1,REINVENT_mean,dis,csk,169670.2,964580.2,0.559826,0.033661,0.000121,0.516835,0.175880,0.013492,13023.4
2,DrugEx_GT_mean,dis,csk,242118.8,891906.4,0.695865,0.028720,0.000154,1.010732,0.271447,0.008831,7851.4
3,DrugEx_GT_epsilon_0.1_mean,dis,csk,140428.0,809233.2,0.545466,0.039156,0.000130,0.511284,0.172969,0.004915,4063.2
4,DrugEx_GT_epsilon_0.6_mean,dis,csk,267648.0,907474.4,0.679649,0.025317,0.000147,1.055034,0.294816,0.007886,7186.4
5,DrugEx_RNN_mean,dis,csk,110595.8,868587.2,0.665922,0.059490,0.000151,0.450804,0.127252,0.018142,16078.4
6,DrugEx_RNN_epsilon_0.1_mean,dis,csk,72000.6,797636.8,0.387368,0.052695,0.000106,0.220224,0.090468,0.005051,3941.0
7,DrugEx_RNN_epsilon_0.6_mean,dis,csk,87100.4,891065.4,0.620167,0.071460,0.000139,0.351509,0.097631,0.017469,15682.2
8,GB_GA_mean,dis,csk,2710.0,9998.0,0.197592,0.744888,0.004049,0.280268,0.271057,0.010821,108.2
9,addcarbon_mean,dis,csk,856.8,1000000.0,0.033403,0.383717,0.000006,0.000148,0.000857,0.001827,1827.0


In [94]:
type_cluster = 'sim' #options: 'dis'|'sim' 
scaffold_type = 'csk' #options: 'csk'|'murcko'
generators_name_list = ['Molpher','REINVENT','DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6','DrugEx_RNN','DrugEx_RNN_epsilon_0.1','DrugEx_RNN_epsilon_0.6', 'GB_GA', 'addcarbon'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,Molpher_mean,sim,csk,103807.6,836443.4,0.698610,0.067420,0.000168,0.471041,0.124157,0.013026,10759.2
1,REINVENT_mean,sim,csk,177692.0,964258.0,0.567807,0.031947,0.000120,0.558746,0.184285,0.012488,12041.2
2,DrugEx_GT_mean,sim,csk,231889.4,759533.4,0.901085,0.051124,0.000358,1.626701,0.313838,0.017673,12369.0
3,DrugEx_GT_epsilon_0.1_mean,sim,csk,136686.8,829696.4,0.611599,0.045393,0.000152,0.573319,0.165093,0.013696,11091.0
4,DrugEx_GT_epsilon_0.6_mean,sim,csk,272645.2,909722.6,0.868628,0.032036,0.000189,1.473212,0.299685,0.017267,15686.4
5,DrugEx_RNN_mean,sim,csk,147793.8,888514.8,0.722764,0.055814,0.000175,0.610825,0.163767,0.019256,16863.6
6,DrugEx_RNN_epsilon_0.1_mean,sim,csk,83562.4,847611.2,0.440370,0.053007,0.000116,0.260740,0.098448,0.020089,17138.8
7,DrugEx_RNN_epsilon_0.6_mean,sim,csk,88952.2,891914.4,0.824861,0.093805,0.000180,0.464722,0.099675,0.025464,22643.4
8,GB_GA_mean,sim,csk,2704.6,9999.6,0.393310,1.481861,0.008029,0.552024,0.270471,0.022080,220.8
9,addcarbon_mean,sim,csk,863.8,1000000.0,0.201548,2.339508,0.000033,0.000759,0.000864,0.014234,14233.8


In [95]:
type_cluster = 'sim' #options: 'dis'|'sim' 
scaffold_type = 'murcko' #options: 'csk'|'murcko'
generators_name_list = ['Molpher','REINVENT','DrugEx_GT', 'DrugEx_GT_epsilon_0.1','DrugEx_GT_epsilon_0.6','DrugEx_RNN','DrugEx_RNN_epsilon_0.1','DrugEx_RNN_epsilon_0.6', 'GB_GA', 'addcarbon'] #options: 'Molpher'|'DrugEx'
receptor = 'Glucocorticoid_receptor' #options: 'Glucocorticoid_receptor'|'Leukocyte_elastase'

df = connect_mean_value(receptor, scaffold_type, type_cluster, generators_name_list)

,name,type_cluster,scaffold,USo,SSo,TUPOR,TUPOR_unique,TUPOR_1,TUPOR_2,SESY,ASER,CwASo
0,Molpher_mean,sim,murcko,215633.0,836455.8,0.321187,0.014887,0.000072,0.182619,0.257852,0.004712,3916.6
1,REINVENT_mean,sim,murcko,633958.2,964342.8,0.117835,0.001847,0.000027,0.182936,0.657409,0.000208,201.0
2,DrugEx_GT_mean,sim,murcko,478566.8,759571.2,0.622711,0.016437,0.000220,0.914102,0.633640,0.004678,3066.0
3,DrugEx_GT_epsilon_0.1_mean,sim,murcko,410774.0,829714.2,0.202269,0.005018,0.000055,0.239582,0.495892,0.008199,6545.2
4,DrugEx_GT_epsilon_0.6_mean,sim,murcko,556595.4,909782.8,0.569102,0.010264,0.000124,0.812021,0.611760,0.004686,4258.2
5,DrugEx_RNN_mean,sim,murcko,499503.4,889313.0,0.402011,0.009020,0.000106,0.528610,0.555117,0.002264,1968.2
6,DrugEx_RNN_epsilon_0.1_mean,sim,murcko,351150.4,847835.6,0.092774,0.002625,0.000025,0.092588,0.413240,0.002329,1933.0
7,DrugEx_RNN_epsilon_0.6_mean,sim,murcko,383463.8,892363.2,0.512333,0.013534,0.000113,0.521756,0.429411,0.003645,3262.2
8,GB_GA_mean,sim,murcko,4381.8,9999.6,0.088792,0.204297,0.001664,0.083721,0.438197,0.008800,88.0
9,addcarbon_mean,sim,murcko,2971.2,1000000.0,0.050297,0.163598,0.000006,0.000267,0.002971,0.003312,3311.6
